In [4]:
### IMPORTACIÓN DE LAS LIBRERÍAS NECESARIAS PARA EJECUTAR NUESTRO CÓDIGO

import os
import requests
import csv
from datetime import datetime
from datetime import timedelta
from bs4 import BeautifulSoup
from dateutil.relativedelta import *

### DEFINICIÓN DE LA FUNCIÓN MEDIANTE LA CUAL SE REALIZARÁ EL WEB SCRAPING

def meteoPozuelo(url,mes_min,mes_max):
    
    mes_min_date   = datetime.strptime(mes_min,"%m/%Y")  ##Mayor o igual a diciembre de 2011

    mes_max_date   = datetime.strptime(mes_max,"%m/%Y")  ##Menor o igual al mes en curso (abril de 2021)

    dd = [] #Se crea una lista vacía donde se recopilará la información que se extrae de la web

    campos = ["Dia","Mes","Año","Temperatura media","Humedad media","Rocío medio","Presión media","Velocidad media",
      "Racha media","Dirección media","Precipitación mensual","Precipitación anual","Intensidad máxima por minuto",
      "Temperatura máxima","Temperatura mínima","Humedad máxima","Humedad mínima","Presión máxima","Presión mínima",
      "Velocidad media máxima","Racha máxima"] #lista con los nombres de los campos a recopilar

    dd.append(campos) #el primer elemento de dd son los nombres de los campos a recopilar
    
    while mes_min_date <= mes_max_date:

        mes_min_YYYYMM = str(mes_min_date.year) + mes_min_date.strftime('%m') 

        url_complet = url + mes_min_YYYYMM

        page = requests.get(url_complet)

        soup = BeautifulSoup(page.content,"html.parser")

        datos = soup.find_all('table')

        datos = datos[:-3] #Eliminamos los tres últimos valores de la lista "datos" porque son tres tablas de datos
                           #globales de todo el mes de temperatura y precipitaciones que no nos interesan en el estudio

        for i in range(len(datos)):   #recorremos cada uno de los días del mes en curso

            dia         = str(datos[i].find_all('tr')[0].td.contents).split()[0][2:]
            mes         = str(datos[i].find_all('tr')[0].td.contents).split()[1]
            año         = mes_min_YYYYMM[:4]
            temp_avg    = str(datos[i].find_all('tr')[1].find_all('td')[1])[5:-5]
            hum_avg     = str(datos[i].find_all('tr')[2].find_all('td')[1])[5:-5]
            roci_avg    = str(datos[i].find_all('tr')[3].find_all('td')[1])[5:-5]
            prec_avg    = str(datos[i].find_all('tr')[4].find_all('td')[1])[5:-5]
            vel_avg     = str(datos[i].find_all('tr')[5].find_all('td')[1])[5:-5]
            rach_avg    = str(datos[i].find_all('tr')[6].find_all('td')[1])[5:-5]
            dir_avg     = str(datos[i].find_all('tr')[7].find_all('td')[1])[5:-5]
            prec_men    = str(datos[i].find_all('tr')[8].find_all('td')[1])[5:-5]
            prec_anu    = str(datos[i].find_all('tr')[9].find_all('td')[1])[5:-5]
            int_max     = str(datos[i].find_all('tr')[10].find_all('td')[1])[5:-5]
            temp_max    = str(datos[i].find_all('tr')[11].find_all('td')[1])[5:-5]
            temp_min    = str(datos[i].find_all('tr')[12].find_all('td')[1])[5:-5]
            hum_max     = str(datos[i].find_all('tr')[13].find_all('td')[1])[5:-5]
            hum_min     = str(datos[i].find_all('tr')[14].find_all('td')[1])[5:-5]
            prec_max    = str(datos[i].find_all('tr')[15].find_all('td')[1])[5:-5]
            prec_min    = str(datos[i].find_all('tr')[16].find_all('td')[1])[5:-5]
            vel_med_max = str(datos[i].find_all('tr')[17].find_all('td')[1])[5:-5]
            rach_max    = str(datos[i].find_all('tr')[18].find_all('td')[1])[5:-5]
            
            dd_diaX = [dia,mes,año,temp_avg,hum_avg,roci_avg,prec_avg,vel_avg,rach_avg,dir_avg,prec_men,prec_anu,
                           int_max,temp_max,temp_min,hum_max,hum_min,prec_max,prec_min,vel_med_max,rach_max]

            dd.append(dd_diaX) 

        mes_min_date = mes_min_date + relativedelta(months = 1)
        
    return dd

###A PARTIR DE ESTE PUNTO SE EJECUTARÁ LA FUNCIÓN Y SE CREARÁ EL FICHERO CSV QUE CONTENDRÁ EL DATASET EXTRAÍDO MEDIANTE WEB SCRAPING

url = "http://www.meteopozuelo.es/wxhistory.php?date="  #URL sin el mes a cargar

mes_min = '01/2018' #Mes mínimo a cargar en formato MM/YYYY

mes_max = '02/2021' #Mes máximo a cargar en formato MM/YYYY

dataset = meteoPozuelo(url,mes_min,mes_max)

directorio_actual = os.getcwd()

nombre_archivo = "datos_meteorologicos_pozuelo_dataset.csv"

ruta_archivo = os.path.join(directorio_actual,nombre_archivo)

with open(ruta_archivo,'w',newline='') as archivo_csv:
    writer = csv.writer(archivo_csv, delimiter=';')
    for i in dataset:
        writer.writerow(i)